In [ ]:
from __future__ import print_function, division
import os
import sys

spark_home = os.environ['SPARK_HOME']
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.4-src.zip'))

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[4]") \
   .appName("test") \
   .enableHiveSupport() \
   .getOrCreate()

sc = spark.sparkContext

# 操作與觀察 Dataframe

## Part1. 基本操作與觀察

### Read file

In [ ]:
fileDF = spark.read.csv("hdfs:///tmp/ratings.csv", sep = ',', header = True)

### 觀察檔案

In [ ]:
fileDF.show()

### 觀察欄位

In [ ]:
fileDF.columns

In [ ]:
len(fileDF.columns)

### 欄位統計值

In [ ]:
fileDF.describe().show()

In [ ]:
fileDF.describe('userid').show()

### 打印 schema

In [ ]:
fileDF.printSchema()

### 選擇欄位

In [ ]:
fileDF.select('userid', 'rating').show()

In [ ]:
fileDF.select('userid', fileDF['rating'] + 1).show()

In [ ]:
fileDF.select('userid', fileDF['rating'] + 1).printSchema()

### 篩選欄位

In [ ]:
fileDF.filter(fileDF['userid'] == 3).show()

In [ ]:
fileDF.filter(fileDF['userid'] == 3).select('userid', 'rating').show()

### 計算不重複值

In [ ]:
fileDF.count()

In [ ]:
fileDF.select('userid').distinct().show()

### 小練習: 有幾部電影被評為5分?

In [ ]:
fileDF.show()

In [ ]:
fileDF.filter(fileDF['rating']==5).select('movieid').distinct().count()

## Part2 資料清理

### 觀察數值分配

In [ ]:
fileDF.crosstab('userid', 'rating').show()

### 處理遺漏值

In [ ]:
fileDF.fillna(0).show()

In [ ]:
fileDF.fillna('0').show()

### 轉換欄位屬性

In [ ]:
from pyspark.sql.types import DoubleType

In [ ]:
fileDF = fileDF.withColumn("rating_double", fileDF["rating"].cast(DoubleType()))

In [ ]:
fileDF.printSchema()

In [ ]:
fileDF.show()

In [ ]:
fileDF.fillna(0).show()

In [ ]:
fileDF_clean = fileDF.fillna(0)

In [ ]:
fileDF_clean.crosstab('userid', 'rating_double').show()

In [ ]:
fileDF.dropna().show()

### 處理重復值

In [ ]:
fileDF.show()

In [ ]:
fileDF.crosstab("userid", "movieid").show()

In [ ]:
(fileDF.select("userid", "movieid", "rating").count() - 
    fileDF.select("userid", "movieid", "rating").distinct().count())

In [ ]:
fileDF.dropDuplicates().orderBy(['userid', 'movieid', 'rating'], ascending=[1,0]).show()

In [ ]:
fileDF.dropDuplicates(['userid', 'movieid', 'rating']).orderBy(['userid', 'movieid', 'rating'], ascending=[1,0]).show()

In [ ]:
fileDF_nodup = fileDF.dropDuplicates(['userid', 'movieid', 'rating']).orderBy(['userid', 'movieid', 'rating'], ascending=[1,0])

In [ ]:
(fileDF_nodup.select("userid", "movieid", "rating").count() - 
    fileDF_nodup.select("userid", "movieid", "rating").distinct().count())

In [ ]:
spark.stop()